<a href="https://colab.research.google.com/github/quang-pham/DeeplearningTutorial/blob/main/%E3%83%87%E3%83%BC%E3%82%BF%E6%8B%A1%E5%BC%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 画像処理用
import cv2
import numpy as np

# ファイル出力用
import os
import shutil
import glob

In [2]:
def  make_image(input_img):
  img_size = input_img.shape
  filter_one = np.ones((3, 3))
    
  # 変換行列(回転の中心, 回転角度, 拡大倍率)
  mat1=cv2.getRotationMatrix2D(tuple(np.array(input_img.shape[:2])/2),20,1)
  mat2=cv2.getRotationMatrix2D(tuple(np.array(input_img.shape[:2])/2),40,1)

  fake_method_array = np.array([
    # 画像の回転(画像データ, 変換行列, 出力画像サイズ)
    lambda image: cv2.warpAffine(image,mat1,image.shape[:2]), 
    lambda image: cv2.warpAffine(image,mat2,image.shape[:2]),

    # しきい値処理(入力画像[グレースケール画像], しきい値, 最大値, 処理内容)
    # 処理内容: cv2.THRESH_TOZERO=しきい値より大きい値は元のまま、それ以外の値が0
    lambda image: cv2.threshold(image, 100, 255, cv2.THRESH_TOZERO)[1],

    # ぼかし(画像, ぼかしカーネル, 横方向の標準偏差。0だと自動的に計算)
    lambda image: cv2.GaussianBlur(image, (5, 5), 0),

    # リサイズ(画像, 変換後の高さ, 変換後の幅)
    lambda image: cv2.resize(cv2.resize(image, (img_size[1] // 5, img_size[0] // 5)),(img_size[1], img_size[0])),

    # 収縮,モルフォロジー変換(畳み込み処理のようにフィルターとのOR変換)
    lambda image: cv2.erode(image, filter_one),

    # 上下反転
    lambda image: cv2.flip(image, 1),
    ])
    
  # 画像変換処理を実行します
  images = []

  for method in fake_method_array:
    faked_img = method(input_img)
    images.append(faked_img)
    
  return images

In [7]:
source_dir = "img_src/*"
output_dir = "img_expansion"

files = glob.glob(source_dir)
for source_image in files:
  print(source_image)
  # 画像データ読み込み
  # target_image = cv2.imread(source_image.split("/")[1]) # 画像のパス
  target_image = cv2.imread(source_image)
  image_name = source_image.split("/")[1].split(".")[0] #画像名

  # 画像の加工
  fake_images = make_image(target_image)

  # 画像を保存するフォルダを作成
  if not os.path.exists(output_dir):
      os.mkdir(output_dir)

  # 処理のリスト
  process_list = [
      "rotate_20",
      "rotate_45",
      "binarization",
      "blur",
      "rezise",
      "erode",
      "flip"
  ]

  # 拡張したものを保存
  for number, img in enumerate(fake_images):
      cv2.imwrite(f"./{output_dir}/{image_name}_{process_list[number]}.jpg", img)


img_src/panda.png
[[[ 44  32  37]
  [ 44  33  35]
  [ 44  34  34]
  ...
  [ 64  67  80]
  [ 64  66  79]
  [ 64  66  78]]

 [[ 44  34  38]
  [ 42  33  36]
  [ 42  32  35]
  ...
  [ 65  67  81]
  [ 65  66  79]
  [ 64  66  78]]

 [[ 43  35  37]
  [ 42  34  36]
  [ 41  32  36]
  ...
  [ 65  67  81]
  [ 65  67  80]
  [ 64  66  78]]

 ...

 [[110 149 155]
  [ 96 151 148]
  [ 86 157 147]
  ...
  [ 33 121  85]
  [ 31 113  82]
  [ 28 105  75]]

 [[116 159 165]
  [102 157 155]
  [ 90 157 150]
  ...
  [ 34 123  86]
  [ 34 119  86]
  [ 31 113  81]]

 [[140 188 192]
  [120 171 174]
  [101 158 158]
  ...
  [ 32 115  81]
  [ 32 115  83]
  [ 31 114  83]]]


In [ ]:
# 作成したファイルをダウンロードできるようにする
shutil.make_archive("/content/img_expansion", "zip", root_dir="/content/img_expansion")

'/content/img_expansion.zip'

In [ ]:
# TorchVisionのtransformの方が簡単に画像の変換を行う事ができる
# https://pystyle.info/pytorch-list-of-transforms/